In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from collections import Counter

In [2]:
import pandas as pd

# Set max_colwidth to None (default)
pd.set_option('display.max_colwidth', None)

# Read the CSV file
dis_sym_data = pd.read_csv("Original_Dataset.csv")

# Display the first few rows of the DataFrame
dis_sym_data.head()


,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fungal infection,itching,skin_rash,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
dis_sym_data.shape

(4920, 18)

In [4]:
dis_sym_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4920 entries, 0 to 4919
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Disease     4920 non-null   object
 1   Symptom_1   4920 non-null   object
 2   Symptom_2   4920 non-null   object
 3   Symptom_3   4920 non-null   object
 4   Symptom_4   4572 non-null   object
 5   Symptom_5   3714 non-null   object
 6   Symptom_6   2934 non-null   object
 7   Symptom_7   2268 non-null   object
 8   Symptom_8   1944 non-null   object
 9   Symptom_9   1692 non-null   object
 10  Symptom_10  1512 non-null   object
 11  Symptom_11  1194 non-null   object
 12  Symptom_12  744 non-null    object
 13  Symptom_13  504 non-null    object
 14  Symptom_14  306 non-null    object
 15  Symptom_15  240 non-null    object
 16  Symptom_16  192 non-null    object
 17  Symptom_17  72 non-null     object
dtypes: object(18)
memory usage: 692.0+ KB


In [5]:
dis_sym_data.describe

<bound method NDFrame.describe of                                       Disease             Symptom_1  \
0                            Fungal infection               itching   
1                            Fungal infection             skin_rash   
2                            Fungal infection               itching   
3                            Fungal infection               itching   
4                            Fungal infection               itching   
...                                       ...                   ...   
4915  (vertigo) Paroymsal  Positional Vertigo              vomiting   
4916                                     Acne             skin_rash   
4917                  Urinary tract infection   burning_micturition   
4918                                Psoriasis             skin_rash   
4919                                 Impetigo             skin_rash   

                  Symptom_2              Symptom_3                  Symptom_4  \
0                 skin_rash   no

In [6]:
dis_sym_data=dis_sym_data.drop_duplicates()

In [7]:
dis_sym_data=dis_sym_data.dropna

In [8]:
# Check the type of dis_sym_data
print(type(dis_sym_data))

# If it's not a DataFrame, try reading the CSV file again
import pandas as pd
dis_sym_data = pd.read_csv("Original_Dataset.csv")

# Then, check its shape
print(dis_sym_data.shape)


<class 'method'>
(4920, 18)


In [9]:
columns_to_check = []
for col in dis_sym_data.columns:
    if col != 'Disease':
        columns_to_check.append(col)

In [10]:
symptoms = dis_sym_data.iloc[:, 1:].values.flatten()
symptoms = list(set(symptoms))

In [11]:
for symptom in symptoms:
    dis_sym_data[symptom] = dis_sym_data.iloc[:, 1:].apply(lambda row: int(symptom in row.values), axis=1)

dis_sym_data_v1 = dis_sym_data.drop(columns=columns_to_check)

In [12]:
dis_sym_data_v1 = dis_sym_data_v1.loc[:, dis_sym_data_v1.columns.notna()]
dis_sym_data_v1.shape

(4920, 132)

In [13]:
dis_sym_data_v1.columns = dis_sym_data_v1.columns.str.strip()
dis_sym_data_v1.columns

Index(['Disease', 'watering_from_eyes', 'throat_irritation',
       'patches_in_throat', 'skin_rash', 'acute_liver_failure',
       'foul_smell_of urine', 'swelled_lymph_nodes', 'dizziness',
       'excessive_hunger',
       ...
       'sunken_eyes', 'obesity', 'swelling_of_stomach',
       'receiving_blood_transfusion', 'irritation_in_anus',
       'distention_of_abdomen', 'silver_like_dusting', 'altered_sensorium',
       'dark_urine', 'knee_pain'],
      dtype='object', length=132)

In [14]:
var_mod = ['Disease']
le = LabelEncoder()
for i in var_mod:
    dis_sym_data_v1[i] = le.fit_transform(dis_sym_data_v1[i])

In [15]:
X = dis_sym_data_v1.drop(columns="Disease")
y = dis_sym_data_v1['Disease']

In [16]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)

In [44]:
def class_algo(model,X_train,y_train,X_test,y_test):
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    accuracy = metrics.accuracy_score(y_pred,y_test)
    print(model_name,'Accuracy : %s', accuracy)

In [47]:
from sklearn.metrics import accuracy_score, precision_score

# Define your classifiers
clfs = {
    'Logistic Regression': {"model": LogisticRegression()},
    'Decision Tree': {"model": tree.DecisionTreeClassifier()},
    'Random Forest': {"model": RandomForestClassifier()},
    'SVM': {"model": SVC(probability=True)},
    'NaiveBayes': {"model": GaussianNB()}
}

def train(clf, X_train, y_train, X_test, y_test):
    model = clf["model"]  # Extract the model from the dictionary
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='micro')  # Choose an appropriate average setting

    
    return accuracy, precision

accuracy_scores = []
precision_scores = []

for name, clf in clfs.items():
    current_accuracy, current_precision = train(clf, X_train, y_train, X_test, y_test)
    
    print("For ", name)
    print("Accuracy - ", current_accuracy)
    print("Precision - ", current_precision)
    
    accuracy_scores.append(current_accuracy)
    precision_scores.append(current_precision)


For  Logistic Regression
Accuracy -  1.0
Precision -  1.0
For  Decision Tree
Accuracy -  1.0
Precision -  1.0
For  Random Forest
Accuracy -  1.0
Precision -  1.0
For  SVM
Accuracy -  1.0
Precision -  1.0
For  NaiveBayes
Accuracy -  1.0
Precision -  1.0


In [54]:
import pickle
pickle.dump(LogisticRegression, open('Specalist.pkl', 'wb'))

In [48]:
doc_data = pd.read_csv("Doctor_Versus_Disease.csv",encoding='latin1', names=['Disease','Specialist'])

In [49]:
doc_data.head(10)

,Disease,Specialist
0,Drug Reaction,Allergist
1,Allergy,Allergist
2,Hypertension,Cardiologist
3,Heart attack,Cardiologist
4,Psoriasis,Dermatologist
5,Chicken pox,Dermatologist
6,Acne,Dermatologist
7,Impetigo,Dermatologist
8,Fungal infection,Dermatologist
9,Hypothyroidism,Endocrinologist


In [50]:
des_data = pd.read_csv("Disease_Description.csv")

In [51]:
des_data.head()

,Disease,Description
0,Drug Reaction,An adverse drug reaction (ADR) is an injury caused by taking medication. ADRs may occur following a single dose or prolonged administration of a drug or result from the combination of two or more drugs.
1,Malaria,An infectious disease caused by protozoan parasites from the Plasmodium family that can be transmitted by the bite of the Anopheles mosquito or by a contaminated needle or transfusion. Falciparum malaria is the most deadly type.
2,Allergy,"An allergy is an immune system response to a foreign substance that's not typically harmful to your body.They can include certain foods, pollen, or pet dander. Your immune system's job is to keep you healthy by fighting harmful pathogens."
3,Hypothyroidism,"Hypothyroidism, also called underactive thyroid or low thyroid, is a disorder of the endocrine system in which the thyroid gland does not produce enough thyroid hormone."
4,Psoriasis,"Psoriasis is a common skin disorder that forms thick, red, bumpy patches covered with silvery scales. They can pop up anywhere, but most appear on the scalp, elbows, knees, and lower back. Psoriasis can't be passed from person to person. It does sometimes happen in members of the same family."


In [53]:
test_col = []
for col in dis_sym_data_v1.columns:
    if col != 'Disease':
        test_col.append(col)


test_data = {}
symptoms = []
predicted = []
def test_input():
    symptoms.clear()
    predicted.clear()
    num_inputs = int(input("Enter the number of symptoms you have: "))
    for i in range(num_inputs):
        user_input = input("Enter Symptoms #{}: ".format(i+1))
        symptoms.append(user_input)
    print("Symptoms you have:", symptoms)
    for column in test_col:
        test_data[column] = 1 if column in symptoms else 0
        test_df = pd.DataFrame(test_data, index=[0])
    print("Predicting Disease based on 6 ML algorithms...")
    for model_name, values in algorithms.items():
    predict_disease = values["model"].predict(test_df)
    predict_disease = le.inverse_transform(predict_disease)  # Correct the indentation here
    predicted.extend(predict_disease)

    disease_counts = Counter(predicted)
    percentage_per_disease = {disease: (count / 6) * 100 for disease, count in disease_counts.items()}
    result_df = pd.DataFrame({"Disease": list(percentage_per_disease.keys()),
                               "Chances": list(percentage_per_disease.values())})
    result_df = result_df.merge(doc_data, on='Disease', how='left')
    result_df = result_df.merge(des_data, on='Disease', how='left')
    return result_df

IndentationError: expected an indented block after 'for' statement on line 22 (3393117708.py, line 23)